In [1]:
library(readxl)
library(dplyr)
library(ggplot2)

customer_r <- read_excel("customer_r.xlsx")
reservation_r <- read_excel("reservation_r.xlsx")
order_info_r <- read_excel("order_info_r.xlsx")
item_r <- read_excel("item_r.xlsx")
colnames(customer_r) <- tolower(colnames(customer_r))
colnames(reservation_r) <- tolower(colnames(reservation_r))
colnames(order_info_r) <- tolower(colnames(order_info_r))
colnames(item_r) <- tolower(colnames(item_r))

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'ggplot2' was built under R version 3.6.3"

In [2]:
# 고객별 스테이크 주문 여부 확인
# 모든 고객의 예약 번호 데이터셋 생성
df_rsv_customer <- reservation_r %>% 
    select(customer_id, reserv_no) %>%    #고객별 모든 예약 번호 선택
    arrange(customer_id, reserv_no)

head(df_rsv_customer)

customer_id,reserv_no
W1327595,2019061801
W1327595,2019071801
W1327803,2019060301
W1327803,2019091506
W1328432,2019060601
W1328432,2019060701


In [3]:
#스테이크 주문예약번호 데이터셋 생성
df_steak_order_rsv_no <- order_info_r %>% 
    filter(item_id == 'M0005') %>%    # 주문이 스테이크면
    mutate(steak_order = 'Y') %>%    # steak_order 열 데이터를 'Y'로 만든다
    arrange(reserv_no)

head(df_steak_order_rsv_no)

order_no,item_id,reserv_no,quantity,sales,steak_order
190603578341100100,M0005,2019060301,4,140000,Y
190606607189610010,M0005,2019060601,4,140000,Y
190616137200510010,M0005,2019061601,3,105000,Y
190708517202310010,M0005,2019070801,2,70000,Y
190710578348100100,M0005,2019071002,3,105000,Y
190718107186310010,M0005,2019071801,4,140000,Y


In [7]:
df_steak_order_1 <- left_join(df_rsv_customer, df_steak_order_rsv_no, by='reserv_no') %>% 
    group_by(customer_id) %>%    # 고객 번호로 그룹화
    mutate(steak_order = ifelse(is.na(steak_order), 'N', 'Y')) %>%    # 주문 여부가 NA이면 N, Y이면 Y로 바꾼다
    summarise(steak_order = max(steak_order)) %>%    #최대값만 취한다
    arrange(customer_id)

df_dpd_var <- df_steak_order_1    #최종 정리된 고객별 스테이크 주문여부
tail(df_dpd_var)    # 종속변수, 최종고객 182명의 스테이크 주문여부 결과확인

customer_id,steak_order
W371982,N
W373103,Y
W373150,N
W374992,N
W375192,Y
W376429,N


In [6]:
#결측치 제거. 성별이 없으면(NA) 고객 번호 제거
df_customer <- customer_r %>% filter(!is.na(sex_code))

#고객 테이블과 에약 테이블 customer_id를 키로 이너 조인
df_table_join_1 <- inner_join(df_customer, reservation_r, by='customer_id')

#df_table_join_1과 주문 테이블의 reserv_no를 키로 이너 조인
df_table_join_2 <- inner_join(df_table_join_1, order_info_r, by='reserv_no')
str(df_table_join_2) # df_table_join_2 테이블 구조 확인

tibble [322 x 19] (S3: tbl_df/tbl/data.frame)
 $ customer_id   : chr [1:322] "W1346506" "W1347648" "W1347756" "W1347984" ...
 $ customer_name : chr [1:322] "고객71" "고객72" "고객73" "고객74" ...
 $ phone_number  : chr [1:322] "010-1111-1181" "010-1111-1182" "010-1111-1183" "010-1111-1184" ...
 $ email         : chr [1:322] "scust71@sfnb.co.kr" "scust72@sfnb.co.kr" "scust73@sfnb.co.kr" "scust74@sfnb.co.kr" ...
 $ first_reg_date: chr [1:322] "19/09/01" "19/09/04" "19/09/05" "19/09/06" ...
 $ sex_code      : chr [1:322] "F" "M" "M" "M" ...
 $ birth         : chr [1:322] "19820922" "19940812" "19931217" "19810822" ...
 $ job           : chr [1:322] "자영업" "학생" NA "자영업" ...
 $ zip_code      : chr [1:322] "122100" "140100" "152100" "130100" ...
 $ reserv_no     : chr [1:322] "2019102002" "2019092301" "2019111301" "2019120201" ...
 $ reserv_date   : chr [1:322] "20191020" "20190923" "20191113" "20191202" ...
 $ reserv_time   : chr [1:322] "1800" "1800" "1200" "1200" ...
 $ branch        : chr [1:322]

In [12]:
#고객 정보, 성별 정보와 방문 횟수, 매출 합을 요약
df_table_join_3 <- df_table_join_2 %>% 
    group_by(customer_id, sex_code, reserv_no, visitor_cnt) %>% 
    summarise(sales_sum = sum(sales)) %>% 
    group_by(customer_id, sex_code) %>% 
    summarise(visit_sum = n_distinct(reserv_no), visitor_sum = sum(visitor_cnt), sales_sum=sum(sales_sum)/1000) %>% 
    arrange(customer_id)
#n_distinct(reserv_no) : reserv_no의 고유 값만 표시한다

df_idp_var <- df_table_join_3    #독립 변수
df_idp_var    #독립 변수 확인

#중복이 아닌 값을 카운팅하는 함수. 여기서는 중복이 아닌 주문 예약 번호, 즉 방문횟수를 세게 된다.
#분석에 필요한 열만 그룹핑해 정리한다


`summarise()` has grouped output by 'customer_id', 'sex_code', 'reserv_no'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'customer_id'. You can override using the `.groups` argument.


customer_id,sex_code,visit_sum,visitor_sum,sales_sum
W1327595,M,2,6,188
W1327803,M,2,6,210
W1328432,M,3,12,246
W1328505,F,5,15,287
W1328786,M,1,1,10
W1328985,M,2,6,178
W1328993,F,1,4,140
W1329560,M,1,2,24
W1329954,M,1,2,48
W1329962,M,2,4,48
